In [14]:
NEO4J_URI="neo4j+s://b28f340f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "a3D2ffVj4HxGLpDnY5hB6H0yp5PzwIhIJbhEKz0wFNs"


In [15]:
import os
os.environ["NEO4J_URL"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [16]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)
graph


/tmp/ipykernel_17478/716556819.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [19]:
##data set movie-->query in neo4j
movie_query="""
LOAD CSV WITH HEADERS FROM
"" as row

MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director IN split(row.director, '|') |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m)
)
FOREACH (actor IN split(row.actors, '|') |
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)
)
FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)
"""

In [20]:
graph.query(movie_query)

[#96D2]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))): OSError('No data')
[#96DE]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-b28f340f-f485.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))): OSError('No data')
Transaction failed and will be retried in 0.9963199994197016s (Failed to read from defunct connection IPv4Address(('si-b28f340f-f485.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))))


ClientError: {code: Neo.ClientError.Statement.ExternalResourceFailed} {message: Invalid URL '': no protocol:  ()}

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")


In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="mistral-saba-24b")
llm

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

In [ ]:
response=chain.invoke({"query":"who was the director of the movie Casino"})
response